# Performing basic uni- and multivariate statistical analsysis of untargeted metabolomics data

**Updated on:** 2023-06-20 17:33 CEST

In this Jupyter Notebook we perform basic explorative uni- and multivariate statistical analyses of an untargeted metabolomics data set, including data cleaning steps, normalization and batch correction.

<div class="alert alert-block alert-info">

**Authors**: Abzer Kelminal (abzer.shah@uni-tuebingen.de), Francesco Russo (frru@ssi.dk), Filip Ottosson (faot@ssi.dk), Madaleine Ernst (maet@ssi.dk), Axel Walter (axel.walter@uni-tuebingen.de), Carolina Gonzalez (cgonzalez7@eafit.edu.co), Efi Kontou (eeko@biosustain.dtu.dk), Judith Boldt (judith.boldt@dsmz.de) <br>

**Input file format**: .csv files or .txt files <br>
**Outputs**: .csv files, .pdf & .svg images  <br>
**Dependencies**: pandas, numpy, glob, os, itertools, plotly, matplotlib, scipy, sklearn, pingouin, skbio, ipyfilechooser, ipywidgets, pynmranalysis

The session info at the end of this notebook gives info about the versions of all the packages used here.
</div>

---
This Notebook can be run with both Jupyter Notebook & Google Colab.

---

<b> Before starting to run this notebook with your own data, remember to save a copy of this notebook in your own Google Drive! Do so by clicking on File &rarr; Save a copy in Drive. You can give whatever meaningful name to your notebook.
This file should be located in a new folder of your Google Drive named 'Colab Notebooks'. You can also download this notebook: File &rarr; Download &rarr; Download .ipynb.</b>

---

<div class="alert alert-block alert-warning">
<b><font size=3> SPECIAL NOTE: Please read the comments before proceeding with the code and let us know if you run into any errors and if you think it could be commented better. We would highly appreciate your suggestions and comments!!</font> </b> </div>

# <font color ='blue'> 1. Introduction </font>
<a id='intro'></a>

<p style='text-align: justify;'> The example files used in this tutorial are part of a study published by <a href="https://doi.org/10.1016/j.chemosphere.2020.129450">Petras and coworkers (2021)</a>. Here, the authors investigated the coastal environments in northern San Diego, USA, after a major rainfall event in winter 2017/2018 to observe the seawater chemotype. The dataset contains surface seawater samples collected (−10 cm) at 30 sites spaced approximately 300 meters apart and 50–100 meters offshore along the San Diego coastline from Torrey Pines State Beach to Mission Bay (Torrey Pines, La Jolla Shores, La Jolla Reefs, Pacific and Mission Beach) at 3 different time points: Dec 2017, Jan 2018 (After a major rainfall, resulting in decreased salinity of water) and Oct 2018. As a result of the study, a huge shift was observed in the seawater's organic matter chemotype after the rainfall. <br>

<p style='text-align: justify;'> Seawater samples collected during October 2018, were not published in the original article, but are added to this tutorial to have increased sample size. The datasets used here can be found in the MassIVE repository: <a href="https://massive.ucsd.edu/ProteoSAFe/dataset.jsp?task=8a8139d9248b43e0b0fda17495387756">MSV000082312</a> and <a href="https://massive.ucsd.edu/ProteoSAFe/dataset.jsp?task=c8411b76f30a4f4ca5d3e42ec13998dc">MSV000085786</a>. The .mzml files
were preprocessed using <a href="http://mzmine.github.io/">MZmine3</a> and the <a href="https://gnps.ucsd.edu/ProteoSAFe/status.jsp?task=cf6e14abf5604f47b28b467a513d3532">feature-based molecular networking workflow in GNPS</a>.</p>

<p style='text-align: justify;'> We initially clean the feature table by batch correction, blank removal, imputation, normalization, and scaling. Then, we perform univariate and multivariate statistical analyses including unsupervised learning methods (e.g., PCoA and clustering) and supervised classification analysis (Random Forest). Each step is discussed in detail in its own sections.</p>

---

# <font color ='blue'> 2. Preliminary setup for the notebook </font>
<a id='Section-2'>

## <font color ='darkblue'> 2.1 Package installation </font>
<a id = 'pkg_install'></a>
Before we start, we need to install all packages, which we need for our analyses and load the installed packages into our session. Since we have many packages for different sections, we install the packages right before the respective sections to reduce the installation time.

In [ ]:
# Install libraries that are not preinstalled
!pip install pandas numpy plotly scikit-learn scikit-bio pingouin kaleido ipyfilechooser nbformat pynmranalysis session_info

<font color="green"><b>TIP:</b> # operator refers to comments describing the code function. Codes beginning with # is "commented out" and it will not run. To run a commented out code, remove the # and run it again.

In [ ]:
# importing necessary modules
import pandas as pd
import numpy as np
import glob
import os
import itertools
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.figure_factory as ff
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.preprocessing import StandardScaler
from scipy.spatial import distance
from sklearn.decomposition import PCA
import pingouin as pg
import skbio # Don't import on Windows!!
from ipyfilechooser import FileChooser
from ipywidgets import interact
import warnings
from pynmranalysis.normalization import PQN_normalization
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import class_weight
from sklearn.metrics import classification_report
import session_info
import shutil

In [ ]:
# Disable warnings for cleaner output, comment out for debugging
warnings.filterwarnings('ignore')

## <font color ='darkblue'> 2.2 Setting a local working directory </font>
<a id = "set_dir"></a>

<p style='text-align: justify;'> When we set a folder (or directory) as the working directory, we can access the files within the folder just by its names without mentioning the entire file path everytime we use it. Also, all the output files will be saved under the working directory. So, before proceeding with the script further, if you are trying to use your own files for the notebook, then please make sure to include all the necessary input files in one local folder and set it as your working directory. </p>

<div class="alert alert-block alert-warning">
<p style='text-align: justify;'> <b>NOTE:</b> When you run the next cell, it will display an output box where you can simply enter the path of the folder containing all your input files in your local computer.
directory For example: D:\User\Project\Test_Data.</div>

It will be set as your working directory and you can access all the files within it. <b> Whenever you see an output box asking for user input, please note, the script will not proceed further without your input. Hence, make sure to run the notebook cell-by-cell instead of running all cells in the notebook simultaneously. </b>

<p style='text-align: justify;'>In Google Colab homepage &rarr; there are 3 icons on the upper left corner. Click on the 3 dots to see the contents of the notebook. To create a folder with your input files, click on the folder icon &rarr; Right-click anywher on the empty space within the left section &rarr; Select 'new folder' &rarr; Copy the path and paste in the output box of next cell </p>

In [ ]:
#enter the directory for the data files:
data_dir = input("Enter the path of the folder with input files:\n")
os.chdir(data_dir)

In [ ]:
#enter the directory for the results:
result_dir = input("Enter the path of the folder for all output files:\n")
os.chdir(result_dir)

---

## <font color ='darkblue'> 2.3 Loading in and exploring the input files </font>
<a name='load_ip'></a>

1) <b>Feature table:</b> A typical output file of an LC-MS/MS metabolomics experiment, containing all mass spectral features (or peaks) with their corresponding relative intensities across samples. The feature table we use in this tutorial was obtained from MZmine3. (Filetype: .csv file) <br>

2) <b>Metadata:</b> An Excel file saved in .txt format that is created by the user, providing additional information about the samples (e.g. sample type, tissue type, species, timepoint of collection etc.) In this tutorial we are using the [metadata format recognized by GNPS workflows](https://ccms-ucsd.github.io/GNPSDocumentation/metadata/). The first column should be named 'filename' and all remaining column headers should be prefixed with ATTRIBUTE_: e.g. ATTRIBUTE_SampleType, ATTRIBUTE_timepoint etc. (Filetype: .txt file) <br>

Feature table and metadata used in this tutorial can be accessed at <a href="https://github.com/Functional-Metabolomics-Lab/Statistical-analysis-of-non-targeted-LC-MSMS-data/tree/main/data">our Functional Metabolomics Git Repository.</a>

<b>To upload files into Google Colab &rarr; Right-click on the folder you created to 'upload' the necessary files from your computer into the cloud session.</b>

<p style='text-align: justify;'> In addition to that, if available, provide the files for molecular annotations such as SIRIUS, CANOPUS and GNPS annotation files. SIRIUS and CANOPUS performs molecular formula prediction and chemical class predictions respectively. GNPS annotation files can be obtained by performing Feature-Based Molecular Networking (FBMN) analysis on the feature table (provided along with its corresponding metadata). <a href="https://gnps.ucsd.edu/ProteoSAFe/status.jsp?task=43cf48acbe24401e84a50ab2069b3d26">The FBMN job is also publicly available.</a> The GNPS annotation files are a result of FBMN.</p>

<p style='text-align: justify;'> To download the FBMN results locally: Go to your <b>MassIVE</b> or <b>GNPS</b> account &rarr; Jobs &rarr; Click on <b>Status</b> of your FBMN Workflow &rarr; Download Cytoscape Data &rarr; A folder will be downloaded. For Ex: "ProteoSAFe-FEATURE-BASED-MOLECULAR-NETWORKING-5877234d-download_cytoscape_data" &rarr; Unzip this folder to access several sub-folders containing files uploaded for FBMN, such as the feature table, metadata table, and graphml file for visualizing the molecular network (found under the folder "gnps_molecular_network_graphml"). FBMN also offers the option to annotate compounds through GNPS spectral library search or an advanced analog search. These annotated files can be found in the sub-folders <b>"DB_result"</b> and <b>"DB_analog_result"</b> (if analog search is performed). During analog search, the method searches for structurally related molecules within the molecular network using a score threshold, such as a minimum cosine score that MS/MS spectra should achieve in spectral matching with MS/MS spectral libraries to be considered an annotation. It's also possible to define the maximum mass shift between the library and putative analogs found (e.g., 100 Da), thus allowing to annotate more compounds. In the given example, GNPS annotation annotated 255 compounds, while analog search annotated 1987 compounds. However, it is important to assess the analog annotations by looking at the cosine score and mirror match accuracy. </p>

[![More on MZmine](https://img.shields.io/badge/More%20on-MZmine-blue)](https://www.nature.com/articles/s41587-023-01690-2)
[![More on GNPS](https://img.shields.io/badge/More%20on-GNPS-informational)](https://www.nature.com/articles/nbt.3597#Abs2)
[![More on FBMN](https://img.shields.io/badge/More%20on-FBMN-blue)](https://www.nature.com/articles/s41592-020-0933-6)
[![More on SIRIUS](https://img.shields.io/badge/More%20on-SIRIUS-blue)](https://boecker-lab.github.io/docs.sirius.github.io/)

### 2.3.1 Loading the data: Use one of the methods
We can load the data files into the script either from the local working directory or from the web using url.

#### a. Loading files from a local folder

Please make sure to include all the necessary input files in the folder you have set as working directory

In [ ]:
#List all files from the local directory:
filenames= os.listdir(data_dir)
filenames= sorted(filenames)
filenames

In [ ]:
# feature quantification table file location
ft_file = filenames[3]
# meta data table file location
md_file = filenames[1]
# optional analog match file
an_file = filenames[2]

# define separators for different input file formats
separators = {"csv": ",", "tsv": "\t", "txt": "\t"}

# read feature table
if ft_file:
    ft = pd.read_csv(os.path.join(data_dir, ft_file), sep = separators[ft_file.split(".")[-1]])
    ft.style.format("{:.5f}")
else:
    print("Please select a feature file and rerun this cell.")
# read metadata table
if md_file:
    md = pd.read_csv(os.path.join(data_dir, md_file), sep = separators[md_file.split(".")[-1]]).set_index("filename")
else:
    print("Please select a metavalue file and rerun this cell.")

if an_file:
    an = pd.read_csv(os.path.join(data_dir, an_file), sep = separators[an_file.split(".")[-1]])
else:
    print("Please select a metavalue file and rerun this cell.")

#### b. Loading files using URL

<p style='text-align: justify;'> In this section, we provide an example of how to retrieve data from a URL. Here, we are accessing the same feature table, metadata, and analog result files directly from our Functional Metabolomics GitHub page and importing them into R. If you are working with your own dataset in a Google Colab environment, you can obtain the file URL by first loading the input files into the Colab workspace, right-clicking on the file, selecting "Copy path", and then replacing the URL in the subsequent cell.</p>

In [ ]:
#Reading the input data using URL
ft_url = 'https://raw.githubusercontent.com/Functional-Metabolomics-Lab/Statistical-analysis-of-non-targeted-LC-MSMS-data/main/data/SD_BeachSurvey_GapFilled_quant.csv'
md_url = 'https://raw.githubusercontent.com/Functional-Metabolomics-Lab/Statistical-analysis-of-non-targeted-LC-MSMS-data/main/data/20221125_Metadata_SD_Beaches_with_injection_order.txt'
an_url = 'https://raw.githubusercontent.com/Functional-Metabolomics-Lab/Statistical-analysis-of-non-targeted-LC-MSMS-data/main/data/DB_analog_result_FBMN.tsv'
ft = pd.read_csv(ft_url)
md = pd.read_csv(md_url, sep = "\t").set_index("filename")
an = pd.read_csv(an_url, sep = "\t")

#### c. Loading files directly from GNPS

One can also load the files directly from the repositories [MassIVE](https://massive.ucsd.edu/ProteoSAFe/static/massive.jsp) or [GNPS](https://gnps.ucsd.edu/ProteoSAFe/static/gnps-splash.jsp). If one has performed FBMN on their feature table, the files (both, input and output files from FBMN) can be accessed by  providing the task ID in the next cell. Task ID can be found by:  Go to your <b>MassIVE</b> or <b>GNPS</b> account &rarr; Jobs &rarr; unique ID is provided for each job in  'Description' column.

<table>
<thead>
<tr><th>Description</th><th>User</th><th>Workflow</th><th>Workflow Version</th><th>Status</th><th>Protected</th><th>Create Time</th><th>Total Size</th><th>Site</th><th>Delete Task</th></tr>
</thead>
<tbody>
<tr><td><font color="red">ID given here</font></td><td>-</td><td>FBMN</td><td>-</td><td>-</td><td>-</td><td>-</td><td>-</td><td>GNPS</td><td>-</td></tr>
</tbody>
</table>

In [ ]:
taskID = "5877234d0a22497eb5ecff7fd53faea5" # Enter the task ID here

In [ ]:
ft_url = os.path.join('https://proteomics2.ucsd.edu/ProteoSAFe/DownloadResultFile?task='+taskID+'&file=quantification_table_reformatted/&block=main')
md_url = os.path.join('https://proteomics2.ucsd.edu/ProteoSAFe/DownloadResultFile?task='+taskID+'&file=metadata_merged/&block=main')
an_url = os.path.join('https://proteomics2.ucsd.edu/ProteoSAFe/DownloadResultFile?task='+taskID+'&file=DB_analogresult/&block=main')

<blockquote>Make sure your metadata has enough columns (ATTRIBUTES) describing your data. The metadata given for FBMN might contain only few columns, however for downstream statistical analysis, one might need more attributes. In such cases, load the metadata file from a local folder</blockquote>

### 2.3.2 Reading the url from options b,c

In [ ]:
ft = pd.read_csv(ft_url)
md = pd.read_csv(md_url, sep = "\t").set_index("filename")
an = pd.read_csv(an_url, sep = "\t")

### 2.3.3 Viewing the input files

Lets check how the data looks, the below lines of code show the first 6 rows of the feature and metadata tables as well as their dimensions (numbers of rows and columns)

In [ ]:
print('Dimension: ',ft.shape) #gets the dimension (number of rows and columns) of ft
ft.head() # gets the first 5 rows of ft

In [ ]:
print('Dimension: ',md.shape)
md.head()

In [ ]:
print('Dimension: ', an.shape)
an.head(n=2)

### 2.3.4 Exploring the metadata

<p style='text-align: justify;'>Before starting with our analysis, we take a look at our metadata. For this purpose, we have created a function. A function is a collection of commands, which takes one or multiple input variables and creates a corresponding output. By creating functions, we avoid having to write big code chunks multiple times. Instead, we can call a sequence of code lines by their function name.</p>
    
<p style='text-align: justify;'><font color="red"> The following cell will not produce any outputs. </font> The outputs will only be produced when we call the function further downstream and give it an input variable. To explore our metadata we define a function called InsideLevels. This function creates a summary table of our metadata, including data types and levels contained in each column.  <font color ="blue"> The input is a metadata table and the output consists of a summary dataframe. </font></p>

In [ ]:
def inside_levels(df):
    # get all the columns (equals all attributes) -> will be number of rows
    levels = []
    types = []
    count = []
    for col in df.columns:
        types.append(type(df[col][0]))
        levels.append(sorted(set(df[col].dropna())))
        tmp = df[col].value_counts()
        count.append([tmp[levels[-1][i]] for i in range(len(levels[-1]))])
    return pd.DataFrame({"ATTRIBUTES": df.columns, "LEVELS": levels, "COUNT":count, "TYPES": types}, index=range(1, len(levels)+1))

Let's have a look at our metadata, with the above defined function InsideLevels.

In [ ]:
len(md.columns)

In [ ]:
inside_levels(md)

<p style='text-align: justify;'> The above table is a summary of our metadata tabel. For example, the 1st row says that there are 2 different types of samples under 'ATTRIBUTE_Sample-Type', namely "Blank" and "Sample". The number of files corresponding to each of these categories is given in the COUNT column. For example, we have 6 files belonging to the "Blank" sample type and 180 files to the "Sample" sample type. </p>

## <font color ='darkblue'> 2.4 Merging annotations with feature table</font>

<div class="alert alert-block alert-info">
    
The first column in feature table: <b>row ID</b> (the unique ID for each detected feature) is given in different columns in different files: <br>
* In DB result and DB analog result files, the row ID is given in the column: <b>#Scan#</b> ('Compound_Name' column has the annotation information)
* For SIRIUS and CANOPUS summary files, the row ID of the feature table is given in the column <b>id</b>. A typical feature id would be: "3_ProjectName_MZmine3_SIRIUS_1_16", where the last number 16 representing the row ID.
    
    </div>

<p style='text-align: justify;'> Annotating features in a feature table is useful for identifying metabolites that correspond to those features. This can help to understand the biological relevance of the features, thereby assisting in the interpretation of our metabolomics data. Here, we will show how to merge ft and an (analog annotation file) based on the above-mentioned columns. You can use the same method to merge SIRIUS, CANOPUS annotations to your feature table as well. This merged table can be used later, when needed. Before merging two dataframes based on certain columns, make sure that the classes of both columns are the same. Although the values are same, but if one column is numeric class and the other is of character class, this might cause unwanted error. </p>

In [ ]:
#checking if both columns are of similar class
ft["row ID"].dtype== an["#Scan#"].dtype

In [ ]:
ft_an = ft.merge(an, left_on= "row ID",  how='left', right_on= "#Scan#", sort=True)
print('Dimension: ', ft_an.shape)
ft_an.head()

## <font color ='darkblue'> 2.5 Arranging metadata and feature table in the same order</font>

<p style='text-align: justify;'> In the next cells, we bring feature table and metadata in the correct format such that the rownames of the metadata and column names of the feature table are the same. Filenames and the order of files need to correspond in both tables, as we will match metadata attributes to the feature table. In that way, both metadata and feature table, can easily be filtered. </p>

In [ ]:
new_md = md.copy() #storing the files under different names to preserve the original files
# remove the (front & tail) spaces, if any present, from the rownames of md
new_md.index = [name.strip() for name in md.index]
# for each col in new_md
# 1) removing the spaces (if any)
# 2) replace the spaces (in the middle) to underscore
# 3) converting them all to UPPERCASE
for col in new_md.columns:
    if new_md[col].dtype == str:
        new_md[col] = [item.strip().replace(" ", "_").upper() for item in new_md[col]]
new_md= new_md.reindex(sorted(new_md.index), axis=0)
print('Dimension: ',new_md.shape)
new_md.head()

In [ ]:
new_ft = ft.copy().sort_values(by=['row ID']) #storing the files under different names to preserve the original files
if 'ft_an' in globals():
  new_ft.index = [f"X{id}_{round(mz, 3)}_{round(rt, 3)}_{ft_an.loc[ft_an['row ID'] == id]['Compound_Name'].to_string(index=False)}" for id, mz, rt in zip(new_ft["row ID"], new_ft["row m/z"], new_ft["row retention time"])]
else:
    # changing the index in feature table to contain m/z and RT information
    new_ft.index = [f"X{id}_{round(mz, 3)}_{round(rt, 3)}" for id, mz, rt in zip(ft["row ID"], ft["row m/z"], ft["row retention time"])]
# drop all columns that are not mzML or mzXML file names
new_ft.drop(columns=[col for col in new_ft.columns if ".mz" not in col], inplace=True)
# remove " Peak area" from column names
new_ft.rename(columns={col: col.replace(" Peak area", "").strip() for col in new_ft.columns}, inplace=True)
# sort column names
new_ft= new_ft.reindex(sorted(new_ft.columns), axis=1)
print('Dimension: ',new_ft.shape)
new_ft.head()

<p style='text-align: justify;'> After adding the annotation information, the feature table "new_ft" now includes the annotation in the row names of the features. This can be beneficial for downstream analysis, such as univariate statistics, where significant features can be easily identified if they are annotated. Similarly, in supervised analysis like Random Forest, the annotated compound driving the classification can be quickly assessed. This will be further highlighted in the respective univariate and multivariate sections. </p>

#ToDo
Picking mzXML columns?

In [ ]:
# check if new_ft column names and md row names are the same
if sorted(new_ft.columns) == sorted(new_md.index):
    print(f"All {len(new_ft.columns)} files are present in both new_md & new_ft.")
else:
    print("Not all files are present in both new_md & new_ft.\n")
    # print the md rows / ft column which are not in ft columns / md rows and remove them
    ft_cols_not_in_md = [col for col in new_ft.columns if col not in new_md.index]
    print(f"These {len(ft_cols_not_in_md)} columns of feature table are not present in metadata table and will be removed:\n{', '.join(ft_cols_not_in_md)}\n")
    new_ft.drop(columns=ft_cols_not_in_md, inplace=True)
    md_rows_not_in_ft = [row for row in new_md.index if row not in new_ft.columns]
    print(f"These {len(md_rows_not_in_ft)} rows of metadata table are not present in feature table and will be removed:\n{', '.join(md_rows_not_in_ft)}\n")
    new_md.drop(md_rows_not_in_ft, inplace=True)

In [ ]:
list(new_ft.columns) == list(new_md.index) #if this returns False it means some files are missing

The output says that all 186 files are present in both new_md & new_ft. Furthermore, metadata filenames and feature table column names are identical, indicating that they are in the same order. If the above lines returns FALSE, it means some files are missing. To check names of the files that are missing we can run the next cell. If everything went well, the next cell should return no output.

In [ ]:
np.setdiff1d(list(new_ft.columns),list(new_md.index)) # if this is empty, no files should be missing or different between the metadata and the matrix

When the above cell returns some filenames, check the corresponding column names in the feature table for spelling mistakes, case-sensitive errors. Reupload the files with correct metadata and rerun the above steps.

In [ ]:
# print(new_ft.columns) # uncomment to check the column names of new_ft

In [ ]:
#checking the dimensions of our new ft and md:
print("The number of rows and columns in our original ft is:", ft.shape,"\n")
print("The number of rows and columns in our new ft is:", new_ft.shape,"\n")
print("The number of rows and columns in our new md is:", new_md.shape)

Notice that the number of columns of the new feature table is the same as the number of rows in our new metadata. Now, we have both our feature table and metadata in the same order.

---

# <font color ='blue'> 3. Data-cleanup </font>

As a first step of our analysis, prior to data cleanup, let's have a look at the data using a simple Principal Coordinate analysis (PCoA), a multidimensional scaling method to see the sample dissimilarities in a simple 2-dimensional plot. You can also using Principal component analysis (PCA). To perform [PCoA](#pcoa), we first transpose the feature table, [scale](#scaling) it and then calculate the distances using Canberra distance metric. The explanation for scaling & PCoA is provided in the respective sections. Hence, we will just proceed with the following cells for performing PCoA.

In [ ]:
ft_t = pd.DataFrame(new_ft).T #transposing the ft
ft_t = ft_t.apply(pd.to_numeric) #converting all values to numeric
np.array_equal(new_md.index,ft_t.index) #should return True now

In [ ]:
raw_fts = pd.DataFrame(StandardScaler().fit_transform(ft_t), index=ft_t.index, columns=ft_t.columns) #scaling ft_t

# compute multi-dimensional scaling on distrance matrix
from sklearn.metrics import pairwise_distances
raw_pcoa = pairwise_distances(raw_fts, metric='braycurtis')

raw_pcoa = distance.squareform(distance.pdist(raw_fts, metric="braycurtis")) # (m x m) distance measure
DM_dist = skbio.stats.distance.DistanceMatrix(raw_pcoa, ids=raw_fts.T.columns)
PCoA = skbio.stats.ordination.pcoa(DM_dist)

In [ ]:
pcoa_df= PCoA.samples
explained_var= PCoA.proportion_explained*100
pcoa_df.head(3)

In [ ]:
print(new_md.columns)

In [ ]:
title = f'Scores plot- Before Data Cleanup'
attribute = "ATTRIBUTE_Sample_Area"
df = pd.merge(pcoa_df[['PC1', 'PC2']], new_md[attribute].apply(str), left_index=True, right_index=True)

fig = px.scatter(df, x='PC1', y='PC2', template='plotly_white', width=600, height=400, color=attribute)

fig.update_layout(font={"color":"grey", "size":12, "family":"Sans"},
                    title={"text":title, 'x':0.2, "font_color":"#3E3D53"},
                    xaxis_title=f'PCo1 {round(explained_var[0], 2)}%',
                    yaxis_title=f'PCo2 {round(explained_var[1], 2)}%')
display(fig)

In [ ]:
fig.write_image(os.path.join(result_dir,"PCoA_plot_before_datacleanup.svg"), "svg") #Saving the plot

As a first step of data-cleanup step, lets merge the metadata and feature table (transposed) together.

In [ ]:
#merging metadata (new_md) and transposed feature table based on the sample names
ft_merged = pd.merge(new_md.reset_index(),ft_t.reset_index(), on= "index", left_on=None, right_on=None) #by.x="filename" picks the filename column of new_md, by.y =0 indicates the rownames of ft_t
ft_merged.head(3)

In [ ]:
ft_merged.to_csv(os.path.join(result_dir,"Ft_md_merged.csv")) # This file can be used for batch correction

<div class="alert alert-block alert-warning">
<b><font size=3> Skip the Batch correction section if you do not have multiple batches !! </font> </b> </div>

## <font color ='darkblue'> 3.1. Batch Correction (Optional) </font>
<a id="batch_corr"></a>

<p style='text-align: justify;'> A 'Batch' is a group of samples processed and analyzed by the same experimental & instrumental conditions in the same short time period. In general, if we have more samples than the tray size, we might measure them as multiple batches or groups. When arranging samples in a batch for measurement, in order to ensure biological diversity within a batch, in addition to our samples of interest, it is advised to have QCs, blanks, and controls (Wehrens et al., 2016). To merge data from these different batches, we must look for batch-effects, both, between the batches and within each batch and correct these effects. <b>But, prior to batch correction on a dataset, we should evaluate the severity of the batch effect and when it is small, it is best to not perform batch correction as this may result in an incorrect estimation of the biological variance in the data. Instead, we should treat the statistical results with caution (Nygaard et al., 2016). For more details, please read the manuscript </b>.</p>

<p style='text-align: justify;'> In this tutorial, the test dataset was utilized to evaluate the chemical impacts of a significant rain event that occurred in northern San Diego, California (USA) during the Winter of 2017/2018. Despite the presence of a "ATTRIBUTE_Batch" column in the metadata, the 3 groups mentioned are not considered as batches due to their distinct collection conditions. The "ATTRIBUTE_time_run" column clearly indicates that the seawater samples were collected and measured at different times during Dec 2017, Jan 2018 (after rainfall), and Oct 2018, respectively. Also, they were collected 'before' and 'after' rainfall. Therefore, searching for inter-batch effects is not meaningful in our example dataset. In terms of intra-batch effect, since the sample dataset does not have QCs, we cannot correct for the intra-batch effect.</p>

<font color="red"> Add a bit about normalization / scaling correcting for batch effects to a certain extent. </font>

Follow the notebook for Batch Correction: [![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Functional-Metabolomics-Lab/Statistical-analysis-of-non-targeted-LC-MSMS-data/blob/main/Individual_Notebooks/R-Notebooks/Batch_Correction.ipynb)

## <font color ='darkblue'> 3.2 Blank removal </font>
<a id="blank_rem"></a>

<p style='text-align: justify;'> Blank samples contain no analytes of interest and consist, for example, of the solvent, matrix, tissue or growth media that was used to prepare or dissolve the samples and analytes of interest. Such as the analytes, the mass spectral features deriving from blank samples are also detected by the LC-MS/MS instrument.
We need to remove these blank features to obtain accurate results.</p>

<p style='text-align: justify;'>To eliminate these blank features, we initially split the feature table into blanks and samples, and then employ a cutoff filter. Next, we compute the average feature intensities for the blanks and samples, and subsequently calculate the ratio of average_feature_blanks to average_feature_sample. We compare this ratio against the user-defined cutoff to determine which features to be removed. When the cutoff is set to 0.3, it implies that for any feature, up to 30% contribution from the blank and 70% from the sample are allowed. Hence any feature with a ratio greater than 0.3 is removed. By using a lower cutoff, such as 10% (0.1), we would demand a greater contribution from the sample (90%) and restrict the blank's contribution to 10%. Raising the cutoff leads to fewer background features being identified and more analyte features being observed. Conversely, lowering the cutoff is more rigorous and leads to the removal of more features.</p>

### 3.2.1 Splitting data into blanks and samples using metadata
<a id="blank_split"></a>

In order to remove blank features from our samples, we first split our feature table into blanks and samples using the metadata and our InsideLevels function created in [Section 2.3.2](#explore_md).

In [ ]:
#If subset_data exists, it will take it as "data", else take new_md as "data"
if 'subset_data' in locals():
    data = subset_data
else:
    data = new_md
display(inside_levels(data))

condition = int(input("Enter the index number of the attribute to split sample and blank: "))
df = pd.DataFrame({"LEVELS": inside_levels(data).iloc[condition-1]["LEVELS"]})
df.index = [*range(1, len(df)+1)]
display(df)

#Among the shown levels of an attribute, select the ones to keep
blank_id = int(input("Enter the index number of your BLANK: "))
print('Your chosen blank is: ', df['LEVELS'][blank_id])

#Splitting the data into blanks and samples based on the metadata
md_blank = data[data[inside_levels(data)['ATTRIBUTES'][condition]] == df['LEVELS'][blank_id]]
blank = new_ft[list(md_blank.index)]
md_samples = data[data[inside_levels(data)['ATTRIBUTES'][condition]] != df['LEVELS'][blank_id]]
samples = new_ft[list(md_samples.index)]

In [ ]:
# Display the chosen blank
print('Dimension: ',blank.shape)
blank.head()

In [ ]:
# Display the chosen samples
print('Dimension: ',samples.shape)
samples.head()

### 3.2.2 Removing blanks
<a id = "removing_blanks"></a>

Now that we have our feature table split into blanks and samples, we can start removing blank features.

**<font color='red'> We use a cutoff of 0.3 </font>**, meaning that in order for a feature to be considered of interest, it needs to have a ratio of average_feature_blanks vs average_feature_sample <30%. <b>In the below cell you can interactively change the threshold to any value between 0.1 and 1. </b>

In [ ]:
blank_removal = samples.copy()
if (input("Do you want to perform Blank Removal- Y/N: ").upper()=="Y"):

    # When cutoff is low, more noise (or background) detected; With higher cutoff, less background detected, thus more features observed
    cutoff = float(input("Enter Cutoff value between 0.1 & 1 (Ideal cutoff range: 0.1-0.3): ")) # (i.e. 10% - 100%). Ideal cutoff range: 0.1-0.3

    # Getting mean for every feature in blank and Samples
    avg_blank = blank.mean(axis=1, skipna=False) # set skipna = False do not exclude NA/null values when computing the result.
    avg_samples = samples.mean(axis=1, skipna=False)

    # Getting the ratio of blank vs samples
    ratio_blank_samples = (avg_blank+1)/(avg_samples+1)

    # Create an array with boolean values: True (is a real feature, ratio<cutoff) / False (is a blank, background, noise feature, ratio>cutoff)
    is_real_feature = (ratio_blank_samples<cutoff)

    # Checking if there are any NA values present. Having NA values in the 4 variables will affect the final dataset to be created
    temp_NA_Count = pd.concat([avg_blank, avg_samples, ratio_blank_samples, is_real_feature],
                            keys=['avg_blank', 'avg_samples', 'ratio_blank_samples', 'bg_bin'], axis = 1)

    print('No. of NA values in the following columns: ')
    display(pd.DataFrame(temp_NA_Count.isna().sum(), columns=['NA']))

    # Calculating the number of background features and features present (sum(bg_bin) equals number of features to be removed)
    print(f"No. of Background or noise features: {len(samples)-sum(is_real_feature)}")
    print(f"No. of features after excluding noise: {sum(is_real_feature)}")

    blank_removal = samples[is_real_feature.values]
    # save to file
    blank_removal.to_csv(os.path.join(result_dir, "Blanks_Removed.csv"))

In [ ]:
print('Dimension: ',blank_removal.shape)
display(blank_removal.head())

In [ ]:
# Metadata without the blanks info
md_samples = new_md[new_md["ATTRIBUTE_Sample.Type"] == "Sample"] #filtering the rows from metadata with the condition = sample
md_samples.head(3)

In [ ]:
blank_removal.to_csv(os.path.join(result_dir, "Blanks_Removed_with_cutoff_"+ str(cutoff) + ".csv"))

## <font color ='darkblue'> 3.3 Imputation </font>
<a id="norm"></a>

<p style='text-align: justify;'> For several reasons, real world datasets might have some missing values in it, in the form of NA, NANs or 0s. Eventhough the gapfilling step of MZmine fills the missing values, we still end up with some missing values or 0s in our feature table. This could be problematic for statistical analysis. </p>
<p style='text-align: justify;'> In order to have a better dataset, we cannot simply discard those rows or columns with missing values as we will lose a chunk of our valuable data. Instead we can try imputing those missing values. Imputation involves replacing the missing values in the data with a meaningful, reasonable guess. There are several methods, such as: </p>

1) Mean imputation (replacing the missing values in a column with the mean or average of the column)
2) Replacing it with the most frequent value
3) Several other machine learning imputation methods such as k-nearest neighbors algorithm(k-NN), Hidden Markov Model(HMM)

Here, first, we use the blank-removed feature table to assess frequencies across relative intensities. The plot from the below cell shows us how many features have which relative intensities. We then create random values between 0 and the minimum value in our blank-removed table and randomly replace all 0s with these random values.

In [ ]:
bins, bins_label, a = [-1, 0, 1, 10], ["-1","0", "1", "10"], 2

while a<=10:
    bins_label.append(np.format_float_scientific(10**a))
    bins.append(10**a)
    a+=1

freq_table = pd.DataFrame(bins_label)
frequency = pd.DataFrame(np.array(np.unique(np.digitize(blank_removal.to_numpy(), bins, right=True), return_counts=True)).T).set_index(0)
freq_table = pd.concat([freq_table,frequency], axis=1).fillna(0).drop(0)
freq_table.columns = ["intensity", "Frequency"]
freq_table["Log(Frequency)"] = np.log(freq_table["Frequency"]+1)

# get the lowest intensity (that is not zero) as a cutoff LOD value
cutoff_LOD = round(blank_removal.replace(0, np.nan).min(numeric_only=True).min())

fig = px.bar(freq_table, x="intensity", y="Log(Frequency)", template="plotly_white",  width=600, height=400)

fig.update_traces(marker_color="#696880")
fig.update_layout(font={"color":"grey", "size":12, "family":"Sans"},
                  title={"text":"Frequency plot - Gap Filled", "x":0.5, "font_color":"#3E3D53"},
                  xaxis_title= "Range")
fig.write_image(os.path.join(result_dir, "frequency_plot.svg"))
fig.show()

The above histogram shows that, in our, blank-removed feature table, there are many zeros present. And no values in the range between 0 to 1E2. The minimum value greater than 0 in our dataframe is in between 1E2 & 1E3 (and that value is 892).

A random number between this minimum value and zero will be used for imputation.

In [ ]:
imputed = blank_removal.copy()
if(input("Do you want to perform Imputation? - Y/N: ").upper()=="Y"):
    #imputed.replace(0, np.random.randint(0, cutoff_LOD), inplace=True)
    imputed = imputed.apply(lambda x: [np.random.randint(0, cutoff_LOD) if v == 0 else v for v in x])
    print('Dimension: ',imputed.shape)
    display(imputed)
    print('Number of missing values after imputation:', display(pd.DataFrame(imputed.isna().sum(), columns=['NA'])))
    # save to file
    imputed.to_csv(os.path.join(result_dir, f"Imputed_QuantTable.csv"))

## <font color ='darkblue'> 3.4 Normalization </font>
<a id="norm"></a>

Normalization is performed to compensate for differences in total metabolite concentrations among samples (Y. Wu & Li,2016). Many normalization techniques can also correct the batch effects, such as those caused by sample pipetting or extraction. Here, we present 2 types of normalization: Total Ion Current(TIC) or (Probabilistic Quotient Normalization) PQN.

### 3.4.1 Total Ion Current (TIC) or sample-centric normalization
<a id="tic"></a>

<p style='text-align: justify;'> TIC is a simple normalization technique that is commonly used as it is easy to implement and computationally inexpensive. It scales the intensities of all features in a sample by the total ion current (or sum) of the sample. TIC normalization is appropriate when the volume of sample injected into the instrument is consistent across all samples, and when the differences in the total ion count among samples are mainly due to differences in the concentration of sample injected. However, TIC has its limitation such as few features with high ion observation can heavily influence the calculation. Also it assumes most metabolites are stable and equally up/down-regulated among samples, which does not hold true in cases like normal vs cancerous tissues comparison. </p>

In [ ]:
normalized = imputed.copy()
if(input("Do you want to perform TIC Normalization? - Y/N: ").upper()=="Y"):
    # Dividing each element of a particular column with its column sum
    normalized_TIC = normalized.apply(lambda x: x/np.sum(x), axis=0)

    # save to file
    normalized_TIC.to_csv(os.path.join(result_dir, "Normalised_TIC_Quant_table.csv"))

    print('Dimension: ', normalized_TIC.shape)
    display(normalized_TIC.head())

### 3.4.2 Probabilistic Quotient Normalization (PQN)
<a id="pqn"></a>

PQN is a statistical method that aims to correct for technical variability in MS-based metabolomics data. The goal of PQN is to adjust for systematic variation between samples while preserving the relative differences in metabolite levels between samples.

PQN involves a four-step process:

1) TIC normalization: Each spectrum is divided by its TIC to adjust for differences in the total signal intensity between samples.
2) Control spectrum calculation: A control spectrum representing the typical profile of metabolite levels across all samplesis calculated. This can be a median spectrum from all samples.
3) Quotient calculation: For each metabolite feature, the ratio (quotient) of the TIC-normalized intensity of the sample to the control spectrum is calculated.
4) Median normalization: The final normalized value for each feature is the median of all the quotients calculated in step 3.

By using the median of the quotients, PQN reduces the impact of any individual features with extremely high or low values that might skew the normalization.

In [ ]:
if(input("Do you want to perform PQN Normalization? - Y/N: ").upper()=="Y"):
    # Dividing each element of a parPQNular column with its column sum
    normalized_PQN = PQN_normalization(normalized ,ref_norm = "median" , verbose=False)

    # save to file
    normalized_PQN.to_csv(os.path.join(result_dir, "Normalised_PQN_Quant_table.csv"))

    print('Dimension: ', normalized_PQN.shape)
    display(normalized_PQN.head())

## <font color ='darkblue'> 3.5 Scaling </font>
<a id="norm"></a>

One can also perform center-scaling after imputation. Scaling is typically done to make sure that the data is centered around 0 and has a consistent spread to adjust for differences in offset between high and low-abundant metabolites, thus leaving only relevant variation for analysis.

In [ ]:
# transposing the imputed table before scaling
transposed = imputed.T
print(f'Imputed feature table rows/columns: {transposed.shape}')
display(transposed.head(3))
# put the rows in the feature table and metadata in the same order
transposed.sort_index(inplace=True)
md_samples.sort_index(inplace=True)

if (md_samples.index == transposed.index).all():
    pass
else:
    print("WARNING: Sample names in feature and metadata table are NOT the same!")

transposed.to_csv(os.path.join(result_dir, "Imputed_QuantTable_transposed.csv"))

In [ ]:
# center and scale filtered data
scaled = pd.DataFrame(StandardScaler().fit_transform(transposed), index=transposed.index, columns=transposed.columns)
scaled.to_csv(os.path.join(result_dir, "Imputed_Scaled_QuantTable.csv"))

# Merge feature table and metadata to one dataframe:
# "how=inner" performs an inner join (only the filenames that appear in md_samples and data are kept)
data = pd.merge(md_samples, scaled, left_index=True, right_index=True, how="inner")
display(data.head())

# <font color ='blue'> 4. Univariate Analysis </font>
<a id="uni"></a>

<p style='text-align: justify;'>Univariate statistics involves analysing "one" variable (or one category) at a time in an attempt to describe the data. In univariate statistics, our null hypothesis H0 states that there is no relationship between different groups or categories. To test this hypothesis, we use statistical tests to either reject (meaning there is a relationship between groups) or accept the null hypothesis (means no relationship). Below here is a list of some parametric and non-parametric tests used for hypothesis testing. In general, parametric test assumes the data to have a normal distribution whereas non-parametric tests have no such assumption about the distribution of the data. </p>

<table>
    <thead>
        <tr><th><font size=3>Parametric Test</font></th>
            <th><font size=3>Non-Parametric test</font></th>
        </tr>
    </thead>
    <tbody>
        <tr><td><font size=3>Paired t-test</font></td>
            <td><font size=3>Wilcoxon Rank sum test</font></td></tr>
        <tr><td><font size=3>Unpaired t-test</font></td>
            <td><font size=3>Mann Whitney U-test</font></td></tr>
        <tr><td><font size=3>One-way ANOVA</font></td>
            <td><font size=3>Kruskal Wallis Test</font></td></tr>
    </tbody>
</table>

In the following section we will use univariate statistical analyses to investigate how the metabolome is influenced by:
*   Sampling site: We will compare seven different sampling areas and investigate if there is a gradual shift in metabolite levels from along the coast.
*   Heavy rainfall: We will compare the metabolite levels before and and after a heavy rainfall in January 2018.

Once again, let's merge metadata and the data to one dataframe.

In [ ]:
Data_merged = pd.merge(md_samples, scaled, left_index=True, right_index=True) #merging metadata with the sclaed data
Data_merged.to_csv(os.path.join(result_dir, "cleaned_Imp_s_metadata_merged.csv"))

In [ ]:
Data_norm_merged = pd.merge(md_samples, normalized_TIC, left_index=True, right_index=True) #merging metadata with the TIC normalized data
Data_norm_merged.to_csv(os.path.join(result_dir, "cleaned_Imp_normTIC_metadata_merged.csv"))

In [ ]:
# make sure that the merging was successful
print(scaled.shape)
print(md_samples.shape)
print(Data_merged.shape)

## <font color ='darkblue'> 4.1 Test for normality </font>
<a id="norm_test"></a>

In order to decide whether to go for parametric or non-parametric tests, we test for normality. Some common methods to test for normality are:
1. Visual representations like histogram, Q–Q Plot
2. Statistical tests such as Shapiro–Wilk test, Kolmogorov–Smirnov test

The null hypothosis(H0) of these statistical tests states that the data has a normal distribution. H0= TRUE if p > 0.05. <a href="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6350423/">Read more about normality tests</a>

Let's start by inspecting a histogram of the first feature in the dataset:

In [ ]:
norm_test = Data_merged
cols= norm_test.columns
filter_col= [col for col in cols if col.startswith("X")] #select all columns that starts with X
norm_test= norm_test[filter_col]
norm_test.iloc[:,0]

In [ ]:
plt.hist(norm_test.iloc[:,0], bins, facecolor='grey', ec="black")
plt.xlabel(norm_test.iloc[:,0].name)
plt.ylabel('Frequency')
plt.show()

Visualization: We can also visualize the same using a quantile-quantile plot (Q-Q plot), which plots the observed sample distribution against a theoretical normal distribution.  For ex: In our case, the length of <code>norm_test[,1]</code> is 180. There are 180 samples (or datapoints) for that particular feature. In a Q-Q plot, we first sort these 180 datapoints and then calculate the quantile for each datapoint. The quantile of a datapoint will describe the number of datapoints that falls under that particular datapoint with respect to total number of datapoints. Then these sample quantiles are plotted against the quantiles obtained using a standard normal distribution. With a Q-Q plot, we can visually see how much a particular feature varies from normality, but it is very subjective. </p>

In [ ]:
import math
from scipy.stats import lognorm
import statsmodels.api as sm

sm.qqplot(norm_test.iloc[:,0], line="s") #add the line with theoretic normal distribution
plt.show()

Interpretation: The plot indicates that the particular feature follows a non-normal distribution. We can also perform a Shapiro-Wilk test on this feature to assess the normality. Here, H0 states that the data has a normal distribution. In a Shapiro-Wilk test, when W=1, it means H0 is true. Also, H0=TRUE when p > 0.05. Howver, in most cases, it is common to have W<1. To see how different the observed distribution from the "empirical" normal distribution, we can look at the p-value.

In [ ]:
from scipy.stats import shapiro
from scipy.stats import lognorm

#perform Shapiro-Wilk test for normality
shapiro(norm_test.iloc[:,0])

In our case, p<0.05 indicates that the variable follows a non-normal distribution. We can now systematically investigate all metabolite features in the dataset for normality:

We can repeat the same steps and test it on a <b>log-transformed data</b>:

In [ ]:
imputed_log= np.log(transposed) #transposing the imputed table
plt.hist(imputed_log.iloc[:,0], bins=6, facecolor='grey', ec="black") # plot histogram of the first feature
plt.xlabel(imputed_log.iloc[:,0].name)
plt.ylabel('Frequency')
plt.show()

sm.qqplot(imputed_log.iloc[:,0], line="s") # plot the Q-Q plot
plt.show()

In [ ]:
#perform Shapiro-Wilk test for normality
shapiro(imputed_log.iloc[:,0])

Here, from the qqplot and histogram, one can see that log-transformed data is close to normality than the scaled data that was used before.

<font color="red"> You can also change the data in the next cell to the <code> norm_TIC</code> or <code> norm_pqn</code> and continue with the next code blocks to test the normality of normalized features. </font>.

Next, we perform shapiro test on each feature and obtain their p-values. These p-values are then corrected for false discovery rate using BH (Benjamini & Hochberg) correction method. When the significance level of p_adj < 0.05, it rejects H0, thus it will be counted as a non-normal distribution, else it follows normal distribution.

In [ ]:
norm_test = Data_merged
cols= norm_test.columns
filter_col= [col for col in cols if col.startswith("X")] #select all columns that starts with X
norm_test= norm_test[filter_col]
uni_data= norm_test
op_shapiro = pd.DataFrame(uni_data.columns) # Get all column to a dataframe called "op_shapiro"
op_shapiro.rename(columns={ op_shapiro.columns[0]: "Metabolites" }, inplace = True) # Convert row "Metabolites" to header and remove the row
op_shapiro

In [ ]:
# Compute Shapiro-Wilk test for each column
p_values = []
for col in uni_data.columns:
    _, p_value = shapiro(uni_data[col])
    p_values.append(p_value)

p_adjusted = sm.stats.fdrcorrection(p_values)[1]
op_shapiro['p_adj'] = p_adjusted #adding a column "p_adj" with corrected p-values. The method used is FDR
op_shapiro['p_adj'] = op_shapiro['p_adj'].astype(float)
op_shapiro

In [ ]:
# Classify distributions as normal or non-normal based on adjusted p-values
op_shapiro['distribution'] = ['Normal' if p_adj >= 0.05 else 'Non-normal' for p_adj in op_shapiro['p_adj']]

# Compute and print number of features with normal and non-normal distribution
num_normal = sum(op_shapiro['distribution'] == 'Normal')
num_non_normal = sum(op_shapiro['distribution'] == 'Non-normal')
print("No. of features with normal distribution:", num_normal)
print("No. of features with non-normal distribution:", num_non_normal)

This shows majority of the features have non-normal distribution.  Also, performing shapiro test with log-transformed data shows that 479 features are normally distributed. However, majoity  of them are still not normal. Thus non-parametric tests might make more sense to our data. These normality tests are particularly more useful for small dataset. When your sample size is large, one can still perform the paramteric tests.

## <font color ='darkblue'> 4.2 ANOVA </font>
<a id="anova"></a>

<p style='text-align: justify;'> We can also perform the parametric test, ANOVA (Analysis of Variance) on our data. Here, we will test whether metabolite levels were different between different sampling sites. Here, the seven different sampling areas will be compared. We  use the function 'aov' to run statistical analyses using ANOVA. ANOVA makes use of variances of different groups to see if they are different from each other. (Variance = SD<sup>2</sup>) </p>
<p style='text-align: justify;'>
<b>H0 = No differences among the groups (their means or Standard deviations)</b>.Using p-value, one can see if the groups are statistically different from one another. When there is a significant difference, F-ratio, another output of ANOVA will be larger and H0 will be rejected. </p>

$$F-statistic = \frac{\text{between-group variance}}{\text{within groups variance}}$$

In [ ]:
# select an attribute to perform ANOVA
anova_attribute = 'ATTRIBUTE_Sample_Area'

In [ ]:
def gen_anova_data(df, columns, groups_col):
    for col in columns:
        result = pg.anova(data=df, dv=col, between=groups_col, detailed=True).set_index('Source')
        p = result.loc[groups_col, 'p-unc']
        f = result.loc[groups_col, 'F']
        meansq = result.loc[groups_col, 'MS']
        yield col, p, f, meansq

dtypes = [('metabolite', 'U100'), ('p', 'f'), ('F', 'f'), ('MS', 'f')]
anova = pd.DataFrame(np.fromiter(gen_anova_data(Data_merged, uni_data.columns, anova_attribute), dtype=dtypes))
anova["Significance"] = ['Non-significant' if p >= 0.05 else 'Significant' for p in anova['p']]
anova

In [ ]:
# add Bonferroni corrected p-values for multiple testing correction
if 'p_bonferroni' not in anova.columns:
    anova.insert(2, 'p_bonferroni', pg.multicomp(anova['p'], method='bonf')[1])
# add significance
if 'significant' not in anova.columns:
    anova.insert(3, 'significant', anova['p_bonferroni'] < 0.05)
# sort by p-value
anova.sort_values('p', inplace=True)
# save ANOVA table
anova.to_csv(os.path.join(result_dir, 'ANOVA_results.csv'))
anova

In [ ]:
print("Total no.of features on which ANOVA test was performed:", len(anova))
print("No.of features that showed significant difference:", len(anova[anova["Significance"]=="Significant"]))
print("No.of features that did not show significant difference:", len(anova[anova["Significance"]=="Non-significant"]))

**Plot ANOVA results**
<p style='text-align: justify;'> The 'anova_out' results are sorted after the p value. To see if there any significant findings, we will use ggplot to visualize results from the ANOVA, with log(F-Statistic values) on the x-axis and -log(p) on the y-axis. Since there are large differences in the F-Statistic and P-values, it is easier to plot their log values. When displaying the names of the top features in the plot, it can  easily get cluttered if we decide to display too many names. So, starting at the top 6 could be a good idea. Using 'geom_text_repel', one can make sure the labels are not overlapping. </p>

In [ ]:
# first plot insignificant features
fig = px.scatter(x=anova[anova['significant'] == False]['F'].apply(np.log),
                y=anova[anova['significant'] == False]['p'].apply(lambda x: -np.log(x)),
                template='plotly_white', width=600, height=600)
fig.update_traces(marker_color="#696880")

# plot significant features
fig.add_scatter(x=anova[anova['significant']]['F'].apply(np.log),
                y=anova[anova['significant']]['p'].apply(lambda x: -np.log(x)),
                mode='markers+text',
                text=anova['metabolite'].iloc[:4],
                textposition='top left', textfont=dict(color='#ef553b', size=7), name='significant')

fig.update_layout(font={"color":"grey", "size":12, "family":"Sans"},
                  title={"text":"ANOVA - FEATURE SIGNIFICANCE", 'x':0.5, "font_color":"#3E3D53"},
                  xaxis_title="log(F)", yaxis_title="-log(p)", showlegend=False)

# save fig as pdf
fig.write_image(os.path.join(result_dir, "plot_ANOVA.pdf"), scale=3) # you can use different file types here e.g. svg, png

fig.show()

In [ ]:
# boxplots with top 4 metabolites from ANOVA
for metabolite in anova.sort_values('p_bonferroni').iloc[:4, 0]:
    fig = px.box(data, x=anova_attribute, y=metabolite, color=anova_attribute)
    fig.update_layout(showlegend=False, title=metabolite, xaxis_title="", yaxis_title="intensity", template="plotly_white", width=500)
    display(fig)

For the top four metabolites, Mission bay is the area that drives the difference between sampling sites, with much higher levels.

## <font color ='darkblue'> 4.3 Tukey's post-hoc test </font>
<a id ="tukey"></a>
As mentioned above, Tukey's post hoc test is a common post-hoc test after a 1-way anova. It also assumes the data to be normally distributed and homoscedastic (having same variances). One we know that there is a significant difference among different sampling sites, we can use tukey-test to calculate, which features show statistically significant differences between 2 sampling sites.


In [ ]:
# functions to run Tukey's and plot results

def tukey_post_hoc_test(anova_attribute, contrasts, metabolites):
    """
    Perform pairwise Tukey test for all metabolites between contrast combinations.

    Args:
        anova_attribute: A string representing the attribute to use in ANOVA.
        contrasts: A list of tuples, where each tuple contains two strings representing the groups to compare.
        metabolites: A list of strings representing the metabolites to test.

    Returns:
        A pandas DataFrame containing the results of the pairwise Tukey test, including the contrast,
        metabolite, absolute value of the metabolite ID, difference between the means, p-value, Bonferroni
        corrected p-value, and significance (True or False).
    """

    # if a single metabolite gets passed make sure to put it in a list
    if isinstance(metabolites, str):
        metabolites = [metabolites]

    def gen_pairwise_tukey(df, contrasts, metabolites):
        """ Yield results for pairwise Tukey test for all metabolites between contrast combinations."""
        for metabolite in metabolites:
            for contrast in contrasts:
                df_for_tukey = df.iloc[np.where(data[anova_attribute].isin([contrast[0], contrast[-1]]))][[metabolite, anova_attribute]]
                pairwise_tukey = pg.pairwise_tukey(df_for_tukey, dv=metabolite, between=anova_attribute)
                yield f'{contrast[0]}-{contrast[1]}', metabolite, int(metabolite.split('_')[0].replace('X', '')), pairwise_tukey['diff'], pairwise_tukey['p-tukey']

    dtypes = [('contrast', 'U100'), ('stats_metabolite', 'U100'), ('stats_ID', 'i'), ('stats_diff', 'f'), ('stats_p', 'f')]
    tukey = pd.DataFrame(np.fromiter(gen_pairwise_tukey(data, contrasts, metabolites), dtype=dtypes))
    # add Bonferroni corrected p-values
    tukey.insert(5, 'stats_p_bonferroni', pg.multicomp(tukey['stats_p'], method='bonf')[1])
    # add significance
    tukey.insert(6, 'stats_significant', tukey['stats_p_bonferroni'] < 0.05)
    # sort by p-value
    tukey.sort_values('stats_p', inplace=True)

    # write output to csv file
    tukey.to_csv(os.path.join(result_dir, 'TukeyHSD_output.csv'))

    return tukey

def plot_tukey(df):

    # create figure
    fig = px.scatter(template='plotly_white', width=600, height=600)

    # plot insignificant values
    fig.add_trace(go.Scatter(x=df[df['stats_significant'] == False]['stats_diff'],
                            y=df[df['stats_significant'] == False]['stats_p'].apply(lambda x: -np.log(x)),
                            mode='markers', marker_color='#696880', name='insignificant'))

    # plot significant values
    fig.add_trace(go.Scatter(x=df[df['stats_significant']]['stats_diff'],
                            y=df[df['stats_significant']]['stats_p'].apply(lambda x: -np.log(x)),
                            mode='markers+text', text=anova['metabolite'].iloc[:4], textposition='top left',
                            textfont=dict(color='#ef553b', size=8), marker_color='#ef553b', name='significant'))

    fig.update_layout(font={"color":"grey", "size":12, "family":"Sans"},
                    title={"text":"TUKEY", 'x':0.5, "font_color":"#3E3D53"},
                    xaxis_title="stats_diff", yaxis_title="-log(p)")

    # save image as pdf
    fig.write_image(os.path.join(result_dir, "TukeyHSD.pdf"), scale=3)

    display(fig)

For the most significant feature from ANOVA:

In [ ]:
contrasts = list(itertools.combinations(set(Data_merged[anova_attribute]), 2)) # all possible combinations
tukey = tukey_post_hoc_test(anova_attribute, contrasts, anova['metabolite'].iloc[0])
display(tukey)

According to the TukeyHSD function description:
- term: shows the regression model.
- contrast: shows the levels that are compared.
- estimate: The estimated difference between the means of the contrast groups.
- null.value: Value to which the estimate is compared.
- adj.p.value: P-value adjusted for multiple comparisons.
- conf.high: Upper bound on the confidence interval for the estimate.
- conf.low: Lower bound on the confidence interval for the estimate.

Here, every possible pair-wise group difference is explored. From ANOVA results, since Mission Bay seemed to differ from other sampling sites for the four most significant metabolites, we could specifically look at the results from comparison between Mission Bay and another sampling site.

In the example below, we look at the differences between Mission Bay and La Jolla Reefs.

In [ ]:
contrasts = [('Mission_Bay', 'La_Jolla Reefs')]
tukey = tukey_post_hoc_test(anova_attribute, contrasts, anova[anova['significant']]['metabolite'])
display(tukey)
plot_tukey(tukey)

In [ ]:
print("Total no.of features on which ANOVA test was performed:", len(tukey))
print("No.of features that showed significant difference:", len(tukey[tukey["stats_significant"]==True]))
print("No.of features that did not show significant difference:", len(tukey[tukey["stats_significant"]==False]))

## <font color ='darkblue'> 4.4 T-tests </font>
<a id ="tukey"></a>

A T-test is commonly used when one has to compare between only two groups. Here, null hypothesis H0 states no difference between the mean of 2 groups. Similar to the F-statistic used by ANOVA, T-tests use T-statistic.


$$\text{T-statistic} = \frac{\text{Mean}_{\text{group}} - \text{Mean}_{\text{population}}}{\text{SD}_{\text{group}} / \sqrt{\text{group size}}}$$


In our dataset, a heavy rainfall in January 2018 could have influenced the metabolome. We will investigate the effect of the rainfall using t-tests. The 2 conditions will be 'Jan-2018' or 'not Jan-2018'

In [ ]:
ttest_attribute = 'ATTRIBUTE_Month'
target_group = 'Jan'

In [ ]:
def gen_ttest_data(df, columns, ttest_attribute, target_group):
    ttest = []
    for col in columns:
        group1 = df[col][df[ttest_attribute]==target_group]
        group2 = df[col][df[ttest_attribute]!=target_group]
        result = pg.ttest(group1, group2)
        result['Metabolite'] = col

        ttest.append(result)

    ttest = pd.concat(ttest).set_index('Metabolite')

    ttest.insert(8, 'p-bonf', pg.multicomp(ttest['p-val'], method='bonf')[1])
    # add significance
    ttest.insert(9, 'Significance', ttest['p-bonf'] < 0.05)

    return ttest

In [ ]:
ttest = gen_ttest_data(data, scaled.columns, ttest_attribute, target_group)
ttest.head(5)

In [ ]:
# Plot T-test

fig = px.scatter(x=ttest['T'],
                y=ttest['p-bonf'].apply(lambda x: -np.log(x)),
                template='plotly_white', width=600, height=600,
                 color=ttest['Significance'].apply(lambda x: str(x)),
                color_discrete_sequence = ['#696880', '#ef553b'])

fig.update_layout(font={"color":"grey", "size":12, "family":"Sans"},
                  title={"text":"T-test - FEATURE SIGNIFICANCE", 'x':0.5, "font_color":"#3E3D53"},
                  xaxis_title="T", yaxis_title="-Log(p)", showlegend=False)

fig.show()

## <font color ='darkblue'> ToDo: 4.5 Kruskal-Wallis </font>
<a name="kr_wallis"></a>

Kruskal-Wallis Test is a non-parametric version of ANOVA. Here, the test does not assume normality of the data. The median of multiple groups are compared to see if they are statistically different from one another. The null hypothesis H0 states no significant difference among different groups. Based on the p value, we decide whether to reject H0 or not. When H0 is rejected, the alternate hypothesis H1 states that atleast one group is statistically different from the others.
<a href="https://statsandr.com/blog/kruskal-wallis-test-nonparametric-version-anova/#introduction">Read more about Kruskal-Wallis test</a>

## <font color ='darkblue'> ToDo: 4.6 Dunn's post hoc test </font>
<a name ="dunn"></a>

If Kruskal_Wallis test shows there is a significant difference among groups, then we can perform post-hoc test. The most common post-hoc test for Kruskal-Wallis is "Dunn Test" where one can perform multiple pairwise comparison to know exactly which groups are significantly different.
Here, in our example, we use Dunn test to calculate, which features show statistically significant differences between individual sampling sites.

# <font color ='blue'> 5. Multivariate analysis </font>
<a name="multi"></a>

## <font color ='darkblue'> 5.1 PCoA PermANOVA </font>
<a id="norm_test"></a>

<p style='text-align: justify;'> <b>Principal coordinates analysis (PCoA)</b> is a metric multidimensional scaling (MDS) method that attempts to represent sample dissimilarities in a low-dimensional space. It converts a distance matrix consisting of pair-wise distances (dissimilarities) across samples into a 2- or 3-D graph. <a href="https://onlinelibrary.wiley.com/doi/10.1002/0470011815.b2a13070">(Gower, 2005)</a></p>
    
<p style='text-align: justify;'> Different distance metrics can be used to calculate dissimilarities among samples (e.g. Euclidean, Canberra, Minkowski). Performing a principal coordinates analysis using the Euclidean distance metric is the same as performing a principal components analysis (PCA). The selection of the most appropriate metric depends on the nature of your data and assumptions made by the metric.</p>

<p style='text-align: justify;'> Within the metabolomics field the Euclidean, Bray-Curtis, Jaccard or Canberra distances are most commonly used. The Jaccard distance is an unweighted metric (presence/absence) whereas Euclidean, Bray-Curtis and Canberra distances take into account relative abundances (weighted). Some metrics may be better suited for very sparse data (with many zeroes) than others. For example, the Euclidean distance metric is not recommended to be used for highly sparse data. </p>

This [video tutorial by StatQuest](https://www.youtube.com/watch?v=GEn-_dAyYME) summarizes nicely the basic principles of PCoA.

In [ ]:
#calculating Principal components
n = 10
pca = PCA(n_components=n)
pca_df = pd.DataFrame(data = pca.fit_transform(scaled), columns = [f'PC{x}' for x in range(1, n+1)])
pca_df.index = md_samples.index
pca_df

In [ ]:
# To get a scree plot showing the variance of each PC in percentage:
percent_variance = np.round(pca.explained_variance_ratio_* 100, decimals =2)

fig_bar = px.bar(x=pca_df.columns, y=percent_variance, template="plotly_white",  width=500, height=400)
fig_bar.update_traces(marker_color="#696880", width=0.5)
fig_bar.update_layout(font={"color":"grey", "size":12, "family":"Sans"},
                    title={"text":"PCA - VARIANCE", 'x':0.5, "font_color":"#3E3D53"},
                    xaxis_title="principal component", yaxis_title="variance (%)")
fig_bar.show()

TODO make the attibute colors work

In [ ]:
@interact(attribute=sorted(md_samples.columns))
def pca_scatter_plot(attribute):
    title = f'PRINCIPLE COMPONENT ANALYSIS'

    df = pd.merge(pca_df[['PC1', 'PC2']], md_samples[attribute].apply(str), left_index=True, right_index=True)

    fig = px.scatter(df, x='PC1', y='PC2', template='plotly_white', width=600, height=400, color=attribute)

    fig.update_layout(font={"color":"grey", "size":12, "family":"Sans"},
                      title={"text":title, 'x':0.2, "font_color":"#3E3D53"},
                      xaxis_title=f'PC1 {round(pca.explained_variance_ratio_[0]*100, 1)}%',
                      yaxis_title=f'PC2 {round(pca.explained_variance_ratio_[1]*100, 1)}%')
    display(fig)

TODO fix the interact thing

In [ ]:
matrices = ['canberra', 'chebyshev', 'correlation', 'cosine', 'euclidean', 'hamming', 'jaccard', 'matching', 'minkowski', 'seuclidean']
@interact(attribute=sorted(md_samples.columns), distance_matrix=matrices)
def pcoa(attribute, distance_matrix):
    # Create the distance matrix from the original data
    distance_matrix = skbio.stats.distance.DistanceMatrix(distance.squareform(distance.pdist(scaled.values, distance_matrix)))
    # perform PERMANOVA test
    permanova = skbio.stats.distance.permanova(distance_matrix, md_samples[attribute])
    permanova['R2'] = 1 - 1 / (1 + permanova['test statistic'] * permanova['number of groups'] / (permanova['sample size'] - permanova['number of groups'] - 1))
    display(permanova)
    # perfom PCoA
    pcoa = skbio.stats.ordination.pcoa(distance_matrix)
    df = pcoa.samples[['PC1', 'PC2']]
    df = df.set_index(md_samples.index)
    df = pd.merge(df[['PC1', 'PC2']], md_samples[attribute].apply(str), left_index=True, right_index=True)

    title = f'PRINCIPLE COORDINATE ANALYSIS'
    fig = px.scatter(df, x='PC1', y='PC2', template='plotly_white', width=600, height=400, color=attribute)

    fig.update_layout(font={"color":"grey", "size":12, "family":"Sans"},
                      title={"text":title, 'x':0.18, "font_color":"#3E3D53"},
                      xaxis_title=f'PC1 {round(pcoa.proportion_explained[0]*100, 1)}%',
                      yaxis_title=f'PC2 {round(pcoa.proportion_explained[1]*100, 1)}%')
    display(fig)

    # To get a scree plot showing the variance of each PC in percentage:
    percent_variance = np.round(pcoa.proportion_explained* 100, decimals =2)

    fig = px.bar(x=[f'PC{x}' for x in range(1, len(pcoa.proportion_explained)+1)], y=percent_variance, template="plotly_white",  width=500, height=400)
    fig.update_traces(marker_color="#696880", width=0.5)
    fig.update_layout(font={"color":"grey", "size":12, "family":"Sans"},
                      title={"text":"PCoA - VARIANCE", 'x':0.5, "font_color":"#3E3D53"},
                      xaxis_title="principal component", yaxis_title="variance (%)")#
    display(fig)


### ToDo: 5.1.1 PERMANOVA

<p style='text-align: justify;'> Permutational multivariate analysis of variance (PERMANOVA) is a non-parametric method for multivariate ANOVA, where P-values are obtained using permutations. The metric was originally developed within the field of ecology <a href ='https://onlinelibrary.wiley.com/doi/full/10.1002/9781118445112.stat07841'>(Anderson, 2008)</a> but is today widely used in other fields, including the microbiome and metabolomics field. PERMANOVA is used to compare groups of samples and it tests whether the centroid and/or the spread of the samples is different among the groups. Here, H0 states no differences among the groups and it is rejected when there is a significance difference among the groups. </p>

<p style='text-align: justify;'> The adonis2() function in the <a href ='https://cran.r-project.org/web/packages/vegan/index.html'>(vegan package)</a> can be used to perform a PERMANOVA. The input is any dissimilarity matrix and the test-statistic retrieved is a multivariate analogue to Fisher's F-ratio as well as an R2 value (Adonis R2). </p>

### ToDo: 5.1.2 Perform PCoA and assess separation using PERMANOVA

<p style='text-align: justify;'> To speed up the analysis and so we don't have to rewrite the entire code when testing different parameters, we can define a function, which will perform a principal coordinates analysis (PCoA) using a distance metric of choice, calculate a PERMANOVA and plot results in a 2-D graph: </p>

## <font color = 'darkblue'> 5.2 Hierarchial Clustering Algorithm</font>
<a name="hca"></a>

We are now ready to perform a cluter analysis. The concept behind hierarchical clustering is to repeatedly combine the two nearest clusters into a larger cluster.

The first step consists of calculating the distance between every pair of observation points and stores it in a matrix;
1. It puts every point in its own cluster;
2. It merges the closest pairs of points according to their distances;
3. It recomputes the distance between the new cluster and the old ones and stores them in a new distance matrix;
4. It repeats steps 2 and 3 until all the clusters are merged into one single cluster. <br>

### 5.2.1 Dendrogram

In [ ]:
fig = ff.create_dendrogram(scaled, labels=list(scaled.index))
fig.update_layout(width=700, height=500, template='plotly_white')

# save image as pdf
fig.write_image(os.path.join(result_dir, "Cluster_Dendrogram.pdf"), scale=3)
fig.show()

### ToDo: 5.2.2 Methods to find the optimum number of clusters

In [ ]:
# SORT DATA TO CREATE HEATMAP

# Compute linkage matrix from distances for hierarchical clustering
linkage_data_ft = linkage(scaled, method='complete', metric='euclidean')
linkage_data_samples = linkage(scaled.T, method='complete', metric='euclidean')

# Create a dictionary of data structures computed to render the dendrogram.
# We will use dict['leaves']
cluster_samples = dendrogram(linkage_data_ft, no_plot=True)
cluster_ft = dendrogram(linkage_data_samples, no_plot=True)

# Create dataframe with sorted samples
ord_samp = scaled.copy()
ord_samp.reset_index(inplace=True)
ord_samp = ord_samp.reindex(cluster_samples['leaves'])
ord_samp.rename(columns={'index': 'Filename'}, inplace=True)
ord_samp.set_index('Filename', inplace=True)

# Create dataframe with sorted features
ord_ft = ord_samp.T.reset_index()
ord_ft = ord_ft.reindex(cluster_ft['leaves'])
ord_ft.rename(columns={'index': 'Feature'}, inplace=True)
ord_ft.set_index('Feature', inplace=True)

## <font color = 'darkblue'> ToDo: 5.3 Heatmaps</font>
<a name="heat_maps"></a>

In [ ]:
#Heatmap
fig = px.imshow(ord_ft,y=list(ord_ft.index), x=list(ord_ft.columns), text_auto=True, aspect="auto",
               color_continuous_scale='PuOr_r', range_color=[-3,3])

fig.update_layout(
    autosize=False,
    width=700,
    height=800)

fig.update_yaxes(visible=False)
fig.update_xaxes(tickangle = 35)

# save image as pdf
fig.write_image(os.path.join(result_dir, "Heatmap.pdf"), scale=3)

fig.show()

#### ToDo: Heatmap with k-means clustering
ComplexHeatmap contains a function to find clusters by using another clustering algorithm called k-means.

# <font color = 'blue'> 6. Supervised learning with Random Forest</font>
<a name="rf"></a>

<p style='text-align: justify;'> Random Forest is a powerful machine learning algorithm used for both classification and regression tasks. It is an ensemble learning method that combines multiple decision trees to improve the accuracy and robustness of the model. Each tree in the forest is constructed using a random subset of features and training data, which helps to reduce overfitting and increase the diversity of the trees. Random Forest can handle multidimensional data with complex interactions and provides feature importance measures, making it a popular choice for many applications in various fields. </p>

In [ ]:
rf_data = Data_merged.copy() # Storing Data_merged after scaling into another variable named rf_data
print(rf_data.shape)
display(rf_data.head())

Printing all the column names that belong to the metadata category:

In [ ]:
display(rf_data.filter(regex='^(?!X)').columns.tolist()) # Display the column names corresponding to the metadata columns

Here, we are trying to see how RF classifies the samples according to different sample area.
In the cell below, enter the column name of the interested attribute to use for the classification.

In [ ]:
interested_attr = 'ATTRIBUTE_Sample_Area' # Define the attribute of interest

For the RF classification we need to transform the location names to numbers using the OrdinalEncoder.

In [ ]:
# Change the values of the attribute of interest from strings to a numerical
enc = OrdinalEncoder()
labels = rf_data[[interested_attr]]
display(labels.value_counts()) # Displays the sample size for each group
labels = enc.fit_transform(labels)
labels = np.array([x[0] + 1 for x in labels])

In [ ]:
# Extract the features (columns starting with X) and their column names
features = rf_data.filter(regex='^X')
feature_names = features.columns.values.tolist()
print(features.shape)
display(features.head())
features = np.array(features)

Generate a training and test set for the RF classification.

In [ ]:
# Split the data into training and test sets
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size=0.25, random_state=123)

print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

The sample size is different among the groups (see above). We therefore need to balance the size.

In [ ]:
# Balance the weights of the attribute of interest to account for unbalanced sample sizes per group
sklearn_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_labels),
    y=train_labels)
weights = {}
for i,w in enumerate(np.unique(train_labels)):
  weights[w] = sklearn_weights[i]
print(weights)

<p style='text-align: justify;'> Performing Random Forest with 500 trees (n_estimators). Generally, a larger number of this variable improves the performance of the model but also increases the computational cost. It is recommended to start with a reasonable number of trees (e.g., 500-1000) and then tune it based on the performance. </p>

<font color="red"><font size=3> Depending on the number of trees and permutations provided, the following cell may take a considerable amount of time to execute !!</font>

In [ ]:
# Set up the random forest classifier with 500 tress, balanded weights, and a random state to make it reproducible
rf = RandomForestClassifier(n_estimators=500, class_weight=weights, random_state=123)
# Fit the classifier to the training set
rf.fit(train_features, train_labels)

In [ ]:
# Use the random forest classifier to predict the sample areas in the test set
predictions = rf.predict(test_features)
print('Classifier mean accuracy score:', round(rf.score(test_features, test_labels)*100, 2), '%.')

In [ ]:
# Report of the accuracy of predictions on the test set
print(classification_report(test_labels, predictions))
# Print the sample areas corresponding to the numbers in the report
for i,cat in enumerate(enc.categories_[0]):
  print(i+1.0, cat)

**To be edited**

<p style='text-align: justify;'>In Random Forest, the OOB (Out-Of-Bag) error is an estimate of the model's prediction error on unseen data. During the training process, each tree in the forest is grown using a bootstrap sample of the original data, leaving out about one-third of the observations on average. These left-out observations are called the OOB samples. The OOB error line in Random Forest provides an estimate of the model's error rate based on the OOB samples. It is a useful metric for evaluating the performance of the model and for comparing different random forest models. A lower OOB error indicates better predictive performance, but it is important to validate the model on a separate test set to obtain a more accurate estimate of its performance on new data. </p>

In [ ]:
# Most important model quality plot
# OOB error lines should flatline. If it doesn't flatline add more trees
rf = RandomForestClassifier(class_weight=weights, warm_start=True, oob_score=True, random_state=123)
errors = []
tree_range = np.arange(1,500,5)
for i in tree_range:
  rf.set_params(n_estimators=i)
  rf.fit(train_features, train_labels)
  errors.append(rf.oob_score_*100)

plt.plot(tree_range, errors)
plt.ylim(0)
plt.xlabel("Trees")
plt.ylabel("Percent Correct")


**Ranking features by importance**:

We can also rank the feature by importance. The top features had a larger impact on determining the prediction of sampling area.

In [ ]:
# Extract the important features in the model
imp = pd.DataFrame(rf.feature_importances_, index=feature_names).sort_values(by=0, ascending=False)
display(imp[:6])

In [ ]:
# Save the features and their importance
imp.to_csv(os.path.join(result_dir,'Importance_features_RF_500trees.csv'))

# <font color = 'blue'>  7. Conclusion </font>
<a name="outro"></a>

In [ ]:
session_info.show() # To see all the information about the current R session

### Getting output files from Google Colab
For Google Collab users, we can zip the result folder which contains all the output files using the next cell and download the zip file directly from the folder "/content/My_TestData" into the local system.

In [ ]:
# Only for Google Colab
output_filename = '../TestData_Workflow_Results' # Specify the name and location of the output directory (here it will be in '/content/TestData_Workflow_Results')
dir_input = result_dir # Specify the directory to be zipped (here the Results directory specified in the beginning)
shutil.make_archive(output_filename, 'zip', dir_input)